<a href="https://colab.research.google.com/github/youngha-Ju/gong_gong_bigdata_mulcam/blob/main/yeongdeungpo_%ED%9B%84%EB%B3%B4%EC%A7%80_%EC%84%A0%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/new_project'

/content/drive/MyDrive/Colab Notebooks/new_project


In [ ]:
!pip install git+https://github.com/pysal/spopt.git

  Cloning https://github.com/pysal/spopt.git to /tmp/pip-req-build-woqjxnb0
  Running command git clone -q https://github.com/pysal/spopt.git /tmp/pip-req-build-woqjxnb0
     |████████████████████████████████| 2.4 MB 32.6 MB/s 
     |████████████████████████████████| 41.2 MB 76 kB/s 
     |████████████████████████████████| 46 kB 3.9 MB/s 
     |████████████████████████████████| 95 kB 3.7 MB/s 
     |████████████████████████████████| 994 kB 27.2 MB/s 
  Created wheel for spopt: filename=spopt-0.1.2+91.g42356de-py3-none-any.whl size=81165 sha256=6e4a30286712aa9b4ea49ad3f6e4e0cd51d1f24dce5382f78b14c7083489e604
  Stored in directory: /tmp/pip-ephem-wheel-cache-c3nh8rnd/wheels/1a/3c/2f/822a93530f5b7bfdf06eeac6b21933664c686d4811720bd4b2
  Created wheel for esda: filename=esda-2.4.1-py3-none-any.whl size=117429 sha256=799be84afa44f57878a11ff235733db549e425b366b5b52745fce73c5ccf1219
  Stored in directory: /root/.cache/pip/wheels/ed/67/c5/4558a178be4c410f341c31f71b5aba9c761a40c9d17dbdfddd
Succe

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from matplotlib import font_manager 
import folium
from folium import GeoJson
from tqdm import tqdm
import math

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MeanShift
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_samples
from sklearn.cluster import estimate_bandwidth
import scipy.cluster.hierarchy as shc

In [ ]:
# !apt-get updata -qq
# !apt-get install fonts-nanum* -qq
# font_manager._rebuild()

In [ ]:
boundary = gpd.read_file('../big/HangJeongDong_ver20210401.geojson')
sang = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202106.csv')
child_zone = pd.read_csv('./보호구역, 보안등/서울특별시_영등포구_어린이보호구역.csv', encoding='cp949')
light = pd.read_csv('./보호구역, 보안등/서울특별시_영등포구_보안등정보.csv', encoding = 'cp949')
school = pd.read_csv('한국교원대학교_초중고등학교위치.csv', encoding='cp949')
smoking_place = pd.read_csv('흡연구역_전처리_수작업포함.csv', encoding='cp949')

In [ ]:
gu = '영등포구'

In [ ]:
light

,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치연도,설치형태,관리기관전화번호,관리기관명,데이터기준일자
0,당산1동-1,1,서울특별시 영등포구 영신로19길30-4,NaN,37.513538,126.904703,NaN,한전주,02-2670-3838,서울특별시 영등포구,2021-04-16
1,당산1동-10,1,서울특별시 영등포구 영신로19길28,NaN,37.513648,126.905005,NaN,건축물,02-2670-3838,서울특별시 영등포구,2021-04-16
2,당산1동-100,1,서울특별시 영등포구 영신로19길17,NaN,37.513834,126.905962,NaN,한전주,02-2670-3838,서울특별시 영등포구,2021-04-16
3,당산1동-100-1,2,서울특별시 영등포구 영신로19길17,NaN,37.513834,126.905962,NaN,한전주,02-2670-3838,서울특별시 영등포구,2021-04-16
4,당산1동-102,1,서울특별시 영등포구 영신로19길17,NaN,37.513834,126.905962,NaN,한전주,02-2670-3838,서울특별시 영등포구,2021-04-16
...,...,...,...,...,...,...,...,...,...,...,...
8792,영등포본동-96,1,서울특별시 영등포구 도림로47가길6-3,NaN,37.497097,126.900475,NaN,건축물,02-2670-3838,서울특별시 영등포구,2021-04-16
8793,영등포본동-97,1,서울특별시 영등포구 대림로40길 25-12,NaN,37.503144,126.898707,NaN,건축물,02-2670-3838,서울특별시 영등포구,2021-04-16
8794,영등포본동-98,1,서울특별시 영등포구 가마산로31길 32-4,NaN,37.501173,126.894066,NaN,건축물,02-2670-3838,서울특별시 영등포구,2021-04-16
8795,영등포본동-99,1,서울특별시 영등포구 도림로39길 7-12,NaN,37.494326,126.896753,NaN,건축물,02-2670-3838,서울특별시 영등포구,2021-04-16


In [ ]:
'''
구내 모든 보안등과 상가를 folium으로 나타내는 코드, 렉이 심함
'''
# mm = folium.Map(location=[37.513538, 126.904703], zoom_start = 14)
# folium.Choropleth(geo_data=boundary[boundary.sggnm == gu], key_on='feature.properties.adm_nm', fill_color='YlGnBu').add_to(mm)

# for _, i in light.iterrows():
#   folium.Circle(location=[i.위도, i.경도], radius=5, color = 'red').add_to(mm)
# for _, i in sang[sang.시군구명 == gu].iterrows():
#   folium.Marker(location=[i.위도, i.경도]).add_to(mm)

'\n구내 모든 보안등과 상가를 folium으로 나타내는 코드, 렉이 심함\n'

In [ ]:
'''
해당 구 경계면
'''
mm = folium.Map(location=[37.513538, 126.904703], zoom_start = 14)
folium.Choropleth(geo_data=boundary[boundary.sggnm == gu], key_on='feature.properties.adm_nm', fill_color='YlGnBu').add_to(mm)
mm

In [ ]:
smoking_place = smoking_place.drop_duplicates(subset=['위도', '경도'], keep = False)

In [ ]:
smoking_place['위도']=smoking_place['위도'].astype(float)
smoking_place['경도']=smoking_place['경도'].astype(float)
smoking_place = smoking_place[smoking_place.자치구명 == gu].reset_index(drop=True)

In [ ]:
boundary = boundary[boundary.sggnm == gu]

In [ ]:
light = light.dropna(axis=0, subset=['위도'])
light = light.drop_duplicates(subset=['위도', '경도']).reset_index(drop=True)

In [ ]:
sang = sang[sang.시군구명 == gu].reset_index(drop=True)

In [ ]:
school[school['소재지도로명주소'].isna() == True]

,학교ID,학교명,학교급구분,설립일자,설립형태,본교분교구분,운영상태,소재지지번주소,소재지도로명주소,시도교육청코드,시도교육청명,교육지원청코드,교육지원청명,생성일자,변경일자,위도,경도,데이터기준일자
2431,B000023066,신우초등학교,초등학교,2021-09-01,공립,본교,운영,경기도 하남시 감일동 82번지 일원,NaN,7530000,경기도교육청,7692000,경기도광주하남교육지원청,2013-11-29,2021-08-18,37.510998,127.160840,2021-09-15
10231,B000026906,판교대장초등학교,초등학교,2021-06-01,공립,본교,운영,경기도 성남시 분당구 대장동 155일원,NaN,7530000,경기도교육청,7551000,경기도성남교육지원청,2013-11-29,2021-06-01,37.368259,127.069684,2021-09-15
10904,B000026907,판교대장중학교,중학교,2021-06-01,공립,본교,운영,경기도 성남시 분당구 대장동 189일원,NaN,7530000,경기도교육청,7551000,경기도성남교육지원청,2013-11-29,2021-06-01,37.368500,127.069555,2021-09-15


In [ ]:
school[school['소재지지번주소'].isna() == True]

,학교ID,학교명,학교급구분,설립일자,설립형태,본교분교구분,운영상태,소재지지번주소,소재지도로명주소,시도교육청코드,시도교육청명,교육지원청코드,교육지원청명,생성일자,변경일자,위도,경도,데이터기준일자
11957,B000030318,초롱초등학교,초등학교,2021-09-01,공립,본교,운영,NaN,경기도 파주시 양지로 101,7530000,경기도교육청,7681000,경기도파주교육지원청,2013-11-29,2021-08-18,37.710002,126.719917,2021-09-15


In [ ]:
# 서울 내 학교의 소재지 주소 Nan 값이 없기 때문에 실행할 필요는 없음.
school.dropna(subset=['소재지지번주소', '소재지도로명주소'], inplace=True)
id = [idx for idx, x in enumerate(school.소재지도로명주소) if gu in x]
school = school.iloc[id, :]
school.reset_index(drop=True, inplace=True)

In [ ]:
# 대다수의 시설에 흡연부스, 흡연구역이 존재하거나, 사유지로 담배를 피는 것을 허용하는 업종은 제외 
# ex) 'PC/오락/당구/볼링등', '경마/경륜/성인오락', '무도/유흥/가무', '연극/영화/극장', '호텔/콘도',
# 모텔/여관/여인숙 근처의 흡연이 만연하므로 제외하지 않음.
# 흡연 구역의 수요자로 볼수 없는 시설 
# ex) '학원-보습교습입시', '유아교육', '학원-어학'
# 소비자의 방문으로 인한 판매가 주를 이루는 것이 아닌, 출장, 온라인으로 서비스를 제공하여 수익을 주로 얻는 자영업종
# ex) '주택수리' / but 회사의 규모가 있는 업종인 경우는 drop하지 않음.
# 중분류를 기준으로 drop
# 각 대분류에 ranking을 매겨 가중치로 사용.
# 관광/여가/오락 - 165, 부동산 - 302, 생활서비스 - 1186, 소매 - 1409, 숙박 - 38, 스포츠 - 46, 음식 - 2381, 학문/교육 - 350
drop_jung = ['PC/오락/당구/볼링등', '경마/경륜/성인오락', '무도/유흥/가무', '연극/영화/극장', '호텔/콘도', '학원-보습교습입시', '유아교육', '학원-어학', '주택수리']
drop_idx = []
for i in drop_jung:
  drop_idx.extend(list(sang[sang.상권업종중분류명 == i].index))
sang.drop(index=drop_idx, inplace=True)

In [ ]:
# 기존 흡연구역 반영 필요
class PRET:
    def __init__(self, cz, sc, li, sm, cr, sg, gu, bd):
        # cz = child_zone, DataFrame
        # sc = school, DataFrame
        # li = light, DataFrame
        # sm = smoking_place, DataFrame
        # cr = meter, int
        # sg = sangga, DataFrame
        # gu = 행정구, str
        # bd = boundary data, (geojson, shp)
        self.cz = cz
        self.sc = sc
        self.li = li
        self.sm = sm
        self.cr = cr
        self.sg = sg
        self.gu = gu
        self.bd = bd

        self.out_of_coverage()
        self.get_in_buffer()
        self.get_in_buffer_count()
        self.sangga_zero_light()
        self.hcluster()
        self.hcluster_center()
        self.min_distance()

    def out_of_coverage(self):

        self.cz_gdf = gpd.GeoDataFrame(data = self.cz, geometry=gpd.points_from_xy(self.cz.경도, self.cz.위도), crs='epsg:4326')
        self.sc_gdf = gpd.GeoDataFrame(data = self.sc, geometry=gpd.points_from_xy(self.sc.경도, self.sc.위도), crs='epsg:4326')
        self.li_gdf = gpd.GeoDataFrame(data = self.li, geometry=gpd.points_from_xy(self.li.경도, self.li.위도), crs='epsg:4326')
        self.sm_gdf = gpd.GeoDataFrame(data = self.sm, geometry=gpd.points_from_xy(self.sm.경도, self.sm.위도), crs='epsg:4326')
        self.sg_gdf = gpd.GeoDataFrame(data = self.sg, geometry=gpd.points_from_xy(self.sg.경도, self.sg.위도), crs='epsg:4326')

        cz_buffer = self.cz_gdf.to_crs(epsg=5186).geometry.buffer(self.cr)
        cz_coverage = cz_buffer.unary_union
        sc_buffer = self.sc_gdf.to_crs(epsg=5186).geometry.buffer(self.cr)
        sc_coverage = sc_buffer.unary_union
        sm_buffer = self.sm_gdf.to_crs(epsg=5186).geometry.buffer(100)
        sm_coverage = sm_buffer.unary_union

        li_out_of_coverage = self.li_gdf.loc[~self.li_gdf['geometry'].to_crs(epsg=5186).apply(lambda x: cz_coverage.contains(x))]
        li_out_of_coverage = li_out_of_coverage.loc[~li_out_of_coverage['geometry'].to_crs(epsg=5186).apply(lambda x: sc_coverage.contains(x))]
        self.looc = li_out_of_coverage.loc[~li_out_of_coverage['geometry'].to_crs(epsg=5186).apply(lambda x: sm_coverage.contains(x))].reset_index(drop=True)

        # m = folium.Map(location=[37.56, 126.99], zoom_start=11)
        # folium.Choropleth(geo_data=self.bd[self.bd.sggnm == self.gu], key_on='feature.properties.adm_nm', fill_color='YlGnBu').add_to(m) #fill_color='YlGnBu', 'PuRd', 'YlOrR
        # for _, row in self.cz_gdf.iterrows():
        #   folium.Marker([row['위도'], row['경도']]).add_to(m)
        # for _, row in self.sc_gdf.iterrows():
        #   folium.Marker([row['위도'], row['경도']]).add_to(m)
        # for _, row in self.sm_gdf.iterrows():
        #   folium.Marker([row['위도'], row['경도']]).add_to(m)

        # GeoJson(cz_buffer).add_to(m)
        # GeoJson(sc_buffer).add_to(m)
        # GeoJson(sm_buffer).add_to(m)
        # m

    def get_in_buffer(self):

        gib_buffer = self.looc.to_crs(epsg=5186).geometry.buffer(self.cr)
        gib_coverage = gib_buffer.unary_union

        self.sioc = self.sg_gdf.loc[self.sg_gdf['geometry'].to_crs(epsg=5186).apply(lambda x: gib_coverage.contains(x))].reset_index(drop=True)

        # m = folium.Map(location=[37.56, 126.99], zoom_start=11)
        # folium.Choropleth(geo_data=bd[bd.sggnm == gu], key_on='feature.properties.adm_nm', fill_color='YlGnBu').add_to(m) #fill_color='YlGnBu', 'PuRd', 'YlOrR
        # for _, row in cd_gdf.iterrows():
        #   folium.Marker([row['위도'], row['경도']]).add_to(m)
        # for _, row in sangga_in_of_coverage.iterrows():
        #   folium.CircleMarker([row['위도'], row['경도']], radius=5, color='red').add_to(m)
        # GeoJson(cd_buffer).add_to(m)

    def get_in_buffer_count(self):

        self.counts = []
        for _, i in tqdm(self.looc.iterrows()):
          i = pd.DataFrame(data = i)
          i = i.T
          i = gpd.GeoDataFrame(i, geometry=i.geometry,crs='epsg:4326')
          gibc_buffer = i.to_crs(epsg=5186).geometry.buffer(100)
          gibc_coverage = gibc_buffer.unary_union

          sangga_in_of_coverage = self.sg_gdf.loc[self.sg_gdf['geometry'].to_crs(epsg=5186).apply(lambda x: gibc_coverage.contains(x))]
          self.counts.append(len(sangga_in_of_coverage))
          self.mc = np.array(self.counts).mean()
          self.hc_n = int(self.sioc.shape[0] / self.mc)
          
    def sangga_zero_light(self):
        sangga_in_of_coverages = []
        for idx, x in tqdm(self.looc.iterrows()):
          x = pd.DataFrame(data=x)
          x = x.T
          x = gpd.GeoDataFrame(x, geometry=x.geometry, crs='epsg:4326')
          x_buffer = x.to_crs(epsg=5186).geometry.buffer(100)
          x_coverage = x_buffer.unary_union

          sangga_in_of_coverage = self.sioc.loc[self.sioc['geometry'].to_crs(epsg=5186).apply(lambda x: x_coverage.contains(x))]
          if len(sangga_in_of_coverage) != 0:
            sangga_in_of_coverages.append(idx)

        # m = folium.Map(location=[37.56, 126.99], zoom_start=11)
        # folium.Choropleth(geo_data=bd[bd.sggnm == gu], key_on='feature.properties.adm_nm', fill_color='YlGnBu').add_to(m) #fill_color='YlGnBu', 'PuRd', 'YlOrR
        # for _, row in cd_gdf.iterrows():
        #   folium.Marker([row['위도'], row['경도']]).add_to(m)
        # for _, row in sangga_in_of_coverage.iterrows():
        #   folium.CircleMarker([row['위도'], row['경도']], radius=5, color='red').add_to(m)
        # GeoJson(cd_buffer).add_to(m)
        self.looc = self.looc.loc[sangga_in_of_coverages, :]

    def hcluster(self):
        hc = AgglomerativeClustering(self.hc_n, affinity='euclidean', linkage='ward') # "euclidean", "l1", "l2", "manhattan", "cosine", or "precomputed"
        self.y_hc = hc.fit_predict(self.looc[['경도', '위도']])
        self.looc['y_hc'] = self.y_hc

    def hcluster_center(self):
        self.center = []
        for i in range(self.hc_n):
          kms = KMeans(n_clusters=1)
          ykms = kms.fit_predict(self.looc[self.looc.y_hc == i][['경도', '위도']])
          self.center.extend(kms.cluster_centers_)
        
    def min_distance(self):
        distances = []
        self.points = []
        for i in self.center:
          dis = 9999
          for idx, o in self.looc[['경도','위도']].iterrows():
            dist = distance(i, o)
            if dis > dist:
              dis = dist
              point = idx
          self.points.append(point)
          distances.append(dis)
        self.looc = self.looc.loc[self.points, :]

In [ ]:
def distance(p1, p2):
  return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

In [ ]:
a = PRET(child_zone, school, light, smoking_place, 300, sang, gu, boundary)


1719it [23:28,  1.22it/s]
1719it [16:17,  1.76it/s]


In [ ]:
looc = a.looc
sioc = a.sioc

In [ ]:
looc

,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치연도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,geometry,y_hc
1258,신길3동-314,1,서울특별시 영등포구 여의대방로53길22,NaN,37.512051,126.922369,NaN,한전주,02-2670-3838,서울특별시 영등포구,2021-04-16,POINT (126.92237 37.51205),0
1173,문래동-70,1,서울특별시 영등포구 선유서로25길20-9,NaN,37.525121,126.883807,NaN,건축물,02-2670-3838,서울특별시 영등포구,2021-04-16,POINT (126.88381 37.52512),1
393,대림1동-47,1,서울특별시 영등포구 영등포로185-8,NaN,37.520724,126.902139,NaN,건축물,02-2670-3838,서울특별시 영등포구,2021-04-16,POINT (126.90214 37.52072),2
1675,영등포본동-637,1,서울특별시 영등포구 대림로29라길10-2,NaN,37.494762,126.895408,NaN,전용주,02-2670-3838,서울특별시 영등포구,2021-04-16,POINT (126.89541 37.49476),3
1463,양평2동-185,1,서울특별시 영등포구 영등포로86길17,NaN,37.511321,126.923120,NaN,한전주,02-2670-3838,서울특별시 영등포구,2021-04-16,POINT (126.92312 37.51132),4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,영등포동-503,1,서울특별시 영등포구 도림로36길3,NaN,37.492385,126.896919,NaN,전용주,02-2670-3838,서울특별시 영등포구,2021-04-16,POINT (126.89692 37.49238),250
352,대림1동-27,1,서울특별시 영등포구 버드나루로7길12-22,NaN,37.524129,126.908911,NaN,한전주,02-2670-3838,서울특별시 영등포구,2021-04-16,POINT (126.90891 37.52413),251
33,당산1동-138,1,서울특별시 영등포구 영신로21-6,NaN,37.513349,126.908191,NaN,전용주,02-2670-3838,서울특별시 영등포구,2021-04-16,POINT (126.90819 37.51335),252
488,대림2동-136,1,서울특별시 영등포구 영등포로29길12,NaN,37.521837,126.900341,NaN,한전주,02-2670-3838,서울특별시 영등포구,2021-04-16,POINT (126.90034 37.52184),253


In [ ]:
looc.to_csv('%s_looc.csv'%(gu))
sioc.to_csv('%s_sioc.csv'%(gu))